In [ ]:
from functools import cache
import numpy as np

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
garden = lines.copy()

In [ ]:
dirs = [(0, 1), (-1, 0), (1, 0), (0, -1)]


def mv_pos(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])


def get_field(pos, field):
    return field[pos[0]][pos[1]]


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def in_bounds_dim(pos, max_y, max_x):
    return pos[0] >= 0 and pos[0] < max_y and pos[1] >= 0 and pos[1] < max_x

In [ ]:
start_pos = (-1, -1)
for i in range(0, len(garden)):
    for j in range(0, len(garden[0])):
        if get_field((i, j), garden) == "S":
            start_pos = (i, j)
start_pos

In [ ]:
@cache
def step_search(pos, moves_left):
    if moves_left == 0:
        return set([pos])

    pos_moves = set()
    for d in dirs:
        next_pos = mv_pos(pos, d)
        if in_bounds(next_pos, garden) and get_field(next_pos, garden) != "#":
            pos_moves = pos_moves.union(step_search(next_pos, moves_left - 1))
    return pos_moves


# len(step_search(start_pos, 64))

part 2

In [ ]:
# Recursion was very slow and blew up the call stack
# So a non-recursive search was needed
# The infinitely repeated board is implemented by computing the position modulo board size and then looking up that value on the original board

max_y = len(garden)
max_x = len(garden[0])

garden_arr = np.array(["."] * (max_y * max_x)).reshape((max_y, max_x))
for i in range(0, len(garden)):
    for j in range(0, len(garden[0])):
        garden_arr[i][j] = garden[i][j]


def pos_mod(pos, max_y, max_x):
    return (pos[0] % max_y, pos[1] % max_x)


def step_search_inf(steps):
    # A set of tuples with positions and steps left
    # if it was already checked from there, no need to do again
    already_looked_from_here = set()
    search_stack = []

    search_stack.append((start_pos, steps))

    reachable_positions = set()

    while len(search_stack) > 0:
        curr_pos_search = search_stack.pop()
        curr_pos = curr_pos_search[0]
        curr_steps = curr_pos_search[1]

        if (curr_pos, curr_steps) in already_looked_from_here:
            continue
        else:
            already_looked_from_here.add((curr_pos, curr_steps))

        if curr_pos_search[1] == 0:
            reachable_positions.add((curr_pos_search[0]))
            continue

        for d in dirs:
            next_pos = mv_pos(curr_pos_search[0], d)
            if get_field(pos_mod(next_pos, max_y, max_x), garden_arr) != "#":
                search_stack.append((next_pos, curr_pos_search[1] - 1))

    return reachable_positions


# print(step_search_inf(10))
len(step_search_inf(65 + 0 * 131))

In [ ]:
# As always, brute force didn't work out
# I am happy with the non-recursive implementation
# But I did not figure out how to handle the actual input and the sheer size
# The property of the actual input with the middle column and middle row being empty eluded me,
# and while I was sure there must be some cycle to spot, I didn't make the connection
# Coming back to the problem after a while with the tips of the repeats and the empty column and rows, the problem was solved

In [ ]:
# The first idea is to see if the growth becomes stable in a way, either directly in the difference between two iterations
# or that the change between two differences becomes stable
# Because the board is repeated, the assumption is that the cycles depend on board size

# this works for example and and real input

# For example, lets take target value of 1000 steps on example input
# which can be written as 10 + (11*90), cycling 90 times (if a cycle exists) with an offset of 10

# iterating through first few steps to look at the change between values and the change of changes between values
# Steps are 10 + (11*x) with x being a step
# After x=4 the diffdiff starts to be the same at 162

# This is the code for the example input
last_val = 0
diff = 0
diffdiff = 0
cycle = 11
for i in range(0, 8):
    x = 10 + (cycle * i)
    il = len(step_search_inf(x))
    temp_diff = il - last_val
    diffdiff = temp_diff - diff
    diff = temp_diff
    last_val = il
    print(f"i={i}, x={x}, il={il}, diff={diff}, diffdiff={diffdiff}")

In [ ]:
x = 90 - 4  # - 4 because its only stable after the 4th step

res = 1853  # First il value before the diffdiff becomes stable
diff_base = 707  # diff value before diffdiff becomes stable
diffdiff = 162  # stable diffdiff

for i in range(0, x):
    res += diff_base + ((i + 1) * diffdiff)
assert res == 668697

In [ ]:
# or if we want to compute 5000 steps:
# 50 + (11*450) -> 450 steps  with 50 offset
last_val = 0
diff = 0
diffdiff = 0
cycle = 11
for i in range(0, 4):
    x = 50 + (cycle * i)
    il = len(step_search_inf(x))
    temp_diff = il - last_val
    diffdiff = temp_diff - diff
    diff = temp_diff
    last_val = il
    print(f"i={i}, x={x}, il={il}, diff={diff}, diffdiff={diffdiff}")


x = 450 - 1  # stable already after i = 1

res = 2406  # First il value before the diffdiff becomes stable
diff_base = 812  # diff value before diffdiff becomes stable
diffdiff = 162  # stable diffdiff

for i in range(0, x):
    res += diff_base + ((i + 1) * diffdiff)
assert res == 16733044

In [ ]:
# Code for the real input
# 65 + (131*202300)
last_val = 0
diff = 0
diffdiff = 0
cycle = 131
for i in range(0, 4):
    x = 65 + (cycle * i)
    il = len(step_search_inf(x))
    temp_diff = il - last_val
    diffdiff = temp_diff - diff
    diff = temp_diff
    last_val = il
    print(f"i={i}, x={x}, il={il}, diff={diff}, diffdiff={diffdiff}")

In [ ]:
x = int((26501365 - 65) / 131) - 1  # stable after i=1

res = 0  # First il value before the diffdiff becomes stable
diff_base = 0  # diff value before diffdiff becomes stable
diffdiff = 0  # stable diffdiff

for i in range(0, x):
    res += diff_base + ((i + 1) * diffdiff)
res

In [15]:
# With the same idea, a quadratic equation can be interpolated, allowing computation of final value directly

from sympy.solvers import solve
from sympy import Symbol

a = Symbol("a")
b = Symbol("b")
c = Symbol("c")
solved = solve(
    [
        a * (0**2) + b * 0 + c - len(step_search_inf(65 + 0 * 131)),
        a * (1**2) + b * 1 + c - len(step_search_inf(65 + 1 * 131)),
        a * (2**2) + b * 2 + c - len(step_search_inf(65 + 2 * 131)),
    ]
)

x = (26501365 - 65) / 131  # 202300
int(solved[a] * x**2 + solved[b] * x + solved[c])